In [3]:
import numpy as np
import json
import pyscf
from pyscf import gto, scf, dft, df

In [4]:
with np.load("../data/Ethanol/input/0_test_data.npz",allow_pickle=True) as f:
    data=f["data"][()]
    print(data.keys())
    #print("data['mol'] 的原始类型：", type(data["mol"]))  # 先看是字符串还是其他类型
    mol=json.loads(data["mol"])  # 如果是字符串，尝试用 json.loads() 解析
    print(mol.keys())
    atoms=mol["_atom"]
    basis=mol["basis"]
    charge=mol["charge"]
    spin=mol["spin"]
    scf_summary=data["scf_summary"]

dict_keys(['name', 'rho_coeff_default', 'mol', 'Ts', 'C_PBE_aux_default', 'scf_summary', 'Tapbe_aux_default', 'X_PBE_aux_default'])
dict_keys(['max_memory', 'charge', 'spin', 'symmetry', 'symmetry_subgroup', 'cart', 'atom', 'basis', 'nucmod', 'ecp', 'nucprop', 'magmom', '_atm', '_bas', '_env', '_ecpbas', 'groupname', 'topgroup', '_symm_orig', '_symm_axes', '_nelectron', '_nao', '_enuc', '_atom', '_basis', '_ecp', '_built', '_pseudo', '_ctx_lock'])


In [5]:
gt_tsxc = 0
gt_etot = sum(data['scf_summary'].values()) - data['scf_summary']['nuc']
gt_coeff = data['rho_coeff_default']
gt_terms = {
    'j': data['scf_summary']['coul'], 'vext': data['scf_summary']['e1'] - data['Ts'],
    'tsxc': gt_tsxc, 'corr': data['Ts'] + data['scf_summary']['exc'] - gt_tsxc
    }

In [6]:
mol = pyscf.gto.Mole.loads(data['mol'])
mol = pyscf.M(atom=mol.atom, basis=mol.basis)

In [7]:
grid = pyscf.dft.gen_grid.Grids(mol)
grid.level = 2
grid.build()

reference_mol = pyscf.M(atom='H 0 0 0; C 0 -0.5 0.5; N 0 0.5 1; O 0 0.5 -0.5; F 0.5 0 0', basis=mol.basis, spin=1)
auxbasis = pyscf.df.aug_etb(reference_mol, 2.5)

auxmol = pyscf.df.addons.make_auxmol(mol, auxbasis=auxbasis)
auxao_values = pyscf.dft.numint.eval_ao(auxmol, grid.coords, deriv=1)

In [8]:
from datagen import SCFRecorder, scf

mf = dft.RKS(mol, xc='GGA_X_PBE, GGA_C_PBE')
recorder = SCFRecorder()
mf.callback = recorder
mf.init_guess = 'minao'
mf.verbose = 10000

mf.max_memory = 51200
mf.conv_tol = 3.7e-5
mf.grids.level = 2
mf.max_cycle = 50
mf.direct_scf_tol = 1e-10 # eri cutoff

print('====== Before Kernel ======')
# mf.kernel()
scf(mf) # call our custom "kernel"/"scf"
print('====== After Kernel ======')

====== Before Kernel ======


******** <class 'pyscf.dft.rks.RKS'> ********
method = RKS
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 3.7e-05
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-10
chkfile to save SCF result = /tmp/tmpsltk3k9l
max_memory 51200 MB (current use 1112 MB)
XC library pyscf.dft.libxc version 7.0.0
    S. Lehtola, C. Steigemann, M. J.T. Oliveira, and M. A.L. Marques.,  SoftwareX 7, 1–5 (2018)
XC functionals = GGA_X_PBE, GGA_C_PBE
    J. P. Perdew, K. Burke, and M. Ernzerhof.,  Phys. Rev. Lett. 77, 3865 (1996)
    J. P. Perdew, K. Burke, and M. Ernzerhof.,  Phys. Rev. Lett. 78, 1396 (1997)
    J. P. Perdew, K. Burke, and M. Ernzerhof.,  Phys. Rev. Lett. 77, 3865 (1996)
    J. P. Perdew, K. Burke, and M. Ernzerhof.,  Phys. Rev. Lett. 78, 1396 (1997)
small_rho_cutoff = 1e-07
cond(S) = 2100.652775419485
Set gradie

In [9]:
recorder.states[-1]['e_tot']

-154.84174712913176

In [10]:
recorder.states[-1].keys()

dict_keys(['e_tot', 'mo_energy', 'mo_occ', 'mo_coeff', 'scf_summary', 'fock', 'side_fock'])

In [11]:
mf.scf_summary

{'nuc': 81.37417182530255,
 'e1': -371.10121622505415,
 'coul': 156.38763666336388,
 'exc': -21.502339013783725}

In [12]:
import functools
import warnings
S_ORBITAL_COMMON_FACTOR = 0.282094791773878143

@functools.lru_cache(1)
def build_1c1e_helper_mol(mol):
    # Build a helper mol with an invalid basis.
    # That is, the helper mol will have only 1 AO for each atom, which is an S orbital with exp=0.
    # (so that the ao will have values of 1 everywhere in space.)
    old_config = pyscf.gto.mole.NORMALIZE_GTO
    pyscf.gto.mole.NORMALIZE_GTO = False
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore', '.*divide by zero.*')
        helper_mol = pyscf.M(atom=mol.atom, basis={'default': [[0, (0.0, 1.0)]]})
    pyscf.gto.mole.NORMALIZE_GTO = old_config
    # helper_mol will have a coeff of 0.0, which need the following fix.
    for bas_id in range(helper_mol.nbas):
        nprim = helper_mol.bas_nprim(bas_id)
        nctr = helper_mol.bas_nctr(bas_id)
        ptr = helper_mol._bas[bas_id, pyscf.gto.mole.PTR_COEFF]
        helper_mol._env[ptr:ptr+nprim*nctr] = 1 / S_ORBITAL_COMMON_FACTOR
    return helper_mol

In [13]:
def get_jaux(mol, auxmol, dm):
    ti = np.tril_indices(mol.nao)
    tw = (np.ones(mol.nao) * 2 - np.eye(mol.nao))[ti]

    dml = dm[ti] * tw

    # int_3c2e = pyscf.df.incore.aux_e2(mol, auxmol, intor='int3c2e', aosym='s2ij')
    # jaux = dml @ int_3c2e

    pmol = pyscf.gto.mole.conc_mol(mol, auxmol)
    jaux_pieces = []
    for l in range(auxmol.nbas):
        shls_slice = (0, mol.nbas, 0, mol.nbas, mol.nbas + l, mol.nbas + l + 1)
        int3c2e = pmol.intor('int3c2e', aosym='s2ij', shls_slice=shls_slice)
        jaux_pieces.append(dml @ int3c2e)
        del int3c2e
    jaux = np.concatenate(jaux_pieces, axis=-1)
    return jaux

@functools.lru_cache(1)
def int1c1e_nuc_analytical(auxmol):
    helper_mol = build_1c1e_helper_mol(auxmol)
    intor = pyscf.gto.mole.intor_cross('int1e_nuc', helper_mol, auxmol)
    idx = [
        auxmol.bas_atom(ibas)
        for ibas in range(auxmol.nbas) for _ in range(auxmol.bas_angular(ibas) * 2 + 1)
    ]
    intor = intor[idx, range(auxmol.nao)]
    intor = intor / 2  # NOTE: not sure why we need this
    return intor

@functools.lru_cache(1)
def int1c1e_int_analytical(auxmol: pyscf.gto.Mole):
    helper_mol = build_1c1e_helper_mol(auxmol)
    intor = pyscf.gto.mole.intor_cross('int1e_ovlp', helper_mol, auxmol)
    intor = intor[0]
    return intor


def compute_coulumb(coeffs, auxao_2c2e):
    dm = coeffs[None] * coeffs[:, None]
    j = (dm * auxao_2c2e).sum() / 2
    return j


def compute_vext(coeffs, auxao_1c1e_nuc):
    return coeffs @ auxao_1c1e_nuc

def get_rho_auxbasis_jfit_coeff(mol, auxmol, dm, scf_summary):
    int_2c2e = auxmol.intor('int2c2e')

    jaux = get_jaux(mol, auxmol, dm)
    real_vext = (dm * mol.intor('int1e_nuc')).sum()

    int_1c1e_nuc = int1c1e_nuc_analytical(auxmol)
    a = np.concatenate([int_2c2e, int_1c1e_nuc[None]], axis=0)
    b = np.concatenate([jaux, real_vext[None]])
    rho_coeff = np.linalg.lstsq(a, b, rcond=None)[0]

    int_1c1e_int = int1c1e_int_analytical(auxmol)
    real_j = scf_summary['coul'] if scf_summary is not None else float('nan')
    rho_aux_j = compute_coulumb(rho_coeff, int_2c2e)
    rho_aux_vext = compute_vext(rho_coeff, int_1c1e_nuc)
    nelec_int = rho_coeff @ int_1c1e_int
    fit_record = {
        'j_error': rho_aux_j - real_j,
        'vext_error': rho_aux_vext - real_vext,
        'norm_factor': mol.nelectron / nelec_int,
    }

    return rho_coeff, fit_record

In [14]:
recorder.states[-1].keys()

dict_keys(['e_tot', 'mo_energy', 'mo_occ', 'mo_coeff', 'scf_summary', 'fock', 'side_fock'])

In [15]:
initguess_mf = dft.RKS(mol, xc='GGA_X_PBE,GGA_C_PBE')
dm_minu1=initguess_mf.make_rdm1(recorder.states[-1]['mo_coeff'], recorder.states[-1]['mo_occ'])

In [16]:
rho_coe1, fit_record1=get_rho_auxbasis_jfit_coeff(mol=mol, auxmol=auxmol, dm=dm_minu1, scf_summary=mf.scf_summary)

In [17]:
rho_coe1.shape

(454,)

In [18]:
gt_coeff.shape

(454,)

In [19]:
gt_coeff[:5]

array([0.03288491, 0.07282601, 0.18046002, 0.62546803, 1.34284988])

In [20]:
rho_coe1[:5]

array([0.03288491, 0.07282601, 0.18046003, 0.62546804, 1.3428499 ])

In [21]:
np.allclose(rho_coe1, gt_coeff,atol=1e-7)

True

In [22]:
np.max(np.abs(rho_coe1 - gt_coeff))

1.699546785666195e-07

In [23]:
print(fit_record1)

{'j_error': 0.01321895685458685, 'vext_error': 8.43556335894391e-11, 'norm_factor': 1.0000323852159416}


In [24]:
rho = auxao_values[0] @ rho_coe1

In [25]:
with np.load("rho_data_ethanol.pbe.npz",allow_pickle=True) as f:
    print(f.keys())
    rho_ref=f["gt_rho"]
    auxao_values_ref=f["auxao_values"]
    coef_ref=f["gt_coeff_cpu"]

KeysView(NpzFile 'rho_data_ethanol.pbe.npz' with keys: gt_rho, grid_weights, auxao_values, gt_coeff_cpu)


In [26]:
np.allclose(rho, rho_ref, atol=1e-7)

True

In [27]:
np.mean(np.abs(rho - rho_ref))

7.747136771473618e-08

In [28]:
np.allclose(auxao_values[0], auxao_values_ref)

True

In [29]:
np.allclose(rho_coe1, coef_ref, atol=1e-7)

True

In [30]:
def build_J_from_rho_coeff(mol, auxmol, rho_coeff):
    # 1) 拼接三心二电子积分 (μν|P) in s2ij 打包
    pmol = pyscf.gto.mole.conc_mol(mol, auxmol)
    nao = mol.nao
    nbas_aux = auxmol.nbas
    ti = np.tril_indices(nao)
    n_pair = ti[0].size

    J_packed = np.zeros(n_pair, dtype=float)
    p0 = 0
    for l in range(nbas_aux):
        # 每个辅助壳一个块，列数=该壳对应的 AO 个数(含角动量展开)
        shls_slice = (0, mol.nbas, 0, mol.nbas, mol.nbas + l, mol.nbas + l + 1)
        int3c2e_blk = pmol.intor('int3c2e', aosym='s2ij', shls_slice=shls_slice)  # shape: (n_pair, nP_blk)
        nP_blk = int3c2e_blk.shape[1]
        # 2) 对应块与 rho_coeff 的该段相乘并累加
        J_packed += int3c2e_blk @ rho_coeff[p0:p0+nP_blk]
        p0 += nP_blk
        del int3c2e_blk

    # 3) 解包成对称矩阵
    J = np.zeros((nao, nao), dtype=float)
    J[ti] = J_packed
    J = J + J.T - np.diag(np.diag(J))
    return J

In [32]:
J = build_J_from_rho_coeff(mol, auxmol, rho_coe1)
coul_from_J = 0.5 * np.sum(dm_minu1 * J)
coul_from_coeff = (rho_coe1[None]*rho_coe1[:,None] * auxmol.intor('int2c2e')).sum()/2
print(coul_from_J, coul_from_coeff, mf.scf_summary['coul'])

156.40085561381116 156.40085562021846 156.38763666336388


In [33]:
J.shape

(108, 108)